# Questioning Barbie and Oppenheimer Through the Use of Agents

In the following notebook we will build an application that queries both the Barbie and Oppenheimer movies Wikipedia pages, as well as their reviews. 

The main focus of this notebook is to showcase a brief introduction to Agents.

## Build 🏗️

There are 3 main tasks in this notebook:

1. Contruct a Barbie retriever
2. Construct an Oppenheimer retriever
3. Combine the two and allow users to query both resources from a single input through the use of Agents

## Ship 🚢

Based on Tuesday's session - construct a Chainlit (or Gradio) application that allows users to interface with the application.

## Share 🚀

Make a social media post about your final application.

### Dependencies

As always, let's start with some dependencies!

In [112]:
!pip install -q -U langchain openai

In [1]:
import getpass
import os
openai_api_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

### LLM 

We will be leveraging OpenAI's `gpt-3.5-turbo` throughout the notebook, and we can keep it consistent throughout!

In [2]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0)

### Data Collection and Transformation

We'll be leveraging the `WikipediaLoader` tool to collect information from Wikipedia. 

Be sure to set the `doc_content_chars_max` parameter so that you capture the *entire* Wikipedia article content.

In [ ]:
!pip install -q -U wikipedia

In [3]:
from langchain.document_loaders import WikipediaLoader, CSVLoader

barbie_wikipedia_docs = WikipediaLoader(query="Barbie (film)", load_max_docs=1, doc_content_chars_max=1_000_000).load()

barbie_csv_docs = CSVLoader(file_path="./barbie_data/barbie.csv", source_column="Review_Url").load()

Since we'll be using same format source documentation separated by topic, we can save ourselves some extra effort and set up our splitters once. 

We're going to leverage the `RecursiveCharacterTextSplitter` again, this time paying close attention to the format our Wikipedia articles and reviews are in so we can be sure to chunk them appropritately. 

> HINT: You can pass a list of separators when you intialize your `RecursiveTextSplitter`! They are acted on in order of element 0 -> element len(list).

RELEVANT DOCS:
- [`RecursiveCharacterTextSplitter`](https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#langchain.text_splitter.RecursiveCharacterTextSplitter)

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

wikipedia_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0,
    length_function = len,
    is_separator_regex= False,
    separators = ["\n==", "\n", " "] # keep headings, then paragraphs, then sentences
)

csv_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50,
    length_function = len,
    is_separator_regex= False,
    separators = ["\n", " "] # keep paragraphs, then sentences
)

chunked_barbie_wikipedia_docs = wikipedia_text_splitter.transform_documents(barbie_wikipedia_docs)
chunked_barbie_csv_docs = csv_text_splitter.transform_documents(barbie_csv_docs)

#### Retrieval and Embedding Strategy

We've already discussed the useful application of `CacheBackedEmbeddings`, so let's do it again!

RELEVANT DOCS:
- [`CacheBackedEmbeddings`](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.cache.CacheBackedEmbeddings.html#langchain-embeddings-cache-cachebackedembeddings)

In [31]:
!pip install -q -U rank_bm25 tiktoken faiss-cpu

In [5]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# set up cached embeddings store
store = LocalFileStore("./shared_cache/")

core_embeddings_model = OpenAIEmbeddings()

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=core_embeddings_model.model
)

We'll implement a `FAISS` vectorstore, and create a retriever from it.

In [6]:
barbie_csv_faiss_retriever = FAISS.from_documents(chunked_barbie_csv_docs, embedder).as_retriever()


There are a number of excellent options to retrieve documents - we'll be looking at an additional example today, which is called the `EnsembleRetriever`.

The method this is using is outlined in [this paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf).

The brief explanation is:

1. We collect results from two different retrieval methods over the same corpus
2. We apply a reranking algorithm to rerank our source documents to be the *most relevant* without losing specific or potentially low-ranked information rich documents
3. We feed the top-k results into the LLM with our query as context.

> HINT: Your weight list should be of type `List[float]` and the `sum(List[float])` should be `1`.

In [100]:
# set up BM25 retriever
barbie_wikipedia_bm25_retriever = BM25Retriever.from_documents(
    chunked_barbie_wikipedia_docs
)
barbie_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
barbie_wikipedia_faiss_store = FAISS.from_documents(
    chunked_barbie_wikipedia_docs, embedder
)

barbie_wikipedia_faiss_retriever = barbie_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
barbie_ensemble_retriever = EnsembleRetriever(
    retrievers=[barbie_wikipedia_bm25_retriever, barbie_wikipedia_faiss_retriever],
    weights=[0.25, 0.75] # should sum to 1
)

#### Retrieval Agent

We can create a simple conversational retrieval Agent by using the built-ins provided by LangChain!

> HINT: Be sure to provide good natural language descriptions of what the tool should be used for to get the best results.

RELEVANT DOCS:
- [`create_retriever_tool`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool)

In [101]:
from langchain.agents.agent_toolkits import create_retriever_tool

barbie_wikipedia_retrieval_tool = create_retriever_tool(
    barbie_ensemble_retriever, 
    "Wikipedia",
    "Searches and returns documents regarding the plot, history, and cast of the Barbie movie"
)

barbie_csv_retrieval_tool = create_retriever_tool(
    barbie_csv_faiss_retriever,
    "PublicReviews",
    "Searches and returns documents regarding public reviews of the Barbie movie"
)

barbie_retriever_tools = [barbie_wikipedia_retrieval_tool, barbie_csv_retrieval_tool]

Now that we've created our tools, we can combined them into an agent!

RELEVANT DOCS:
- [`create_conversational_retrieval_agent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent.html#langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent)

In [102]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

barbie_retriever_agent_executor = create_conversational_retrieval_agent(llm, barbie_retriever_tools, verbose=True)

In [15]:
barbie_retriever_agent_executor({"input" : "Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?"})



> Entering new AgentExecutor chain...

Invoking: `PublicReviews` with `Barbie movie`


[Document(page_content="Review: The big summer blockbuster of 2023 is a wonderful visual spectacle with a great deal of creative physical comedy, centered in and around the universe of Barbie, Mattel's historic toy doll. Unfortunately, the film tries to be and do much more than what the filmmakers ultimately are able to control and reel in. That a movie about Barbie would be guilty of Americanization is no surprise, and in many ways more than befitting, but the Americanization here isn't just about the history of the toy itself and all its cultural impacts - Barbie also views everything through the eyes of Hollywood anno 2023 - which arguably is a very particular and perhaps not that representative outlook. Instead of utilizing the fine foundation laid in the film's first third, director and co-writer Greta Gervig (Lady Bird, Little Women) gets herself lost in a trite, divisive and most likely futi

{'input': 'Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?',
 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'PublicReviews', 'arguments': '{\n  "__arg1": "Barbie movie"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content="Review: The big summer blockbuster of 2023 is a wonderful visual spectacle with a great deal of creative physical comedy, centered in and around the universe of Barbie, Mattel\'s historic toy doll. Unfortunately, the film tries to be and do much more than what the filmmakers ultimately are able to control and reel in. That a movie about Barbie would be guilty of Americanization is no surprise, and in many ways more than befitting, but the American

In [16]:
barbie_retriever_agent_executor({"input" : "What is a very quick summary of the plot of the Barbie movie?"})



> Entering new AgentExecutor chain...

Invoking: `Wikipedia` with `Barbie movie`


[Document(page_content='Barbie is a 2023 American fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the Barbie fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film follows Barbie (Margot Robbie) and Ken (Ryan Gosling) on a journey of self-discovery following an existential crisis. It features an ensemble supporting cast, including America Ferrera, Kate McKinnon, Helen Mirren, Issa Rae, Simu Liu,', metadata={'title': 'Barbie (film)', 'summary': 'Barbie is a 2023 American fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the Barbie fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film follows Barbie (Margot Robbie) and Ken (Ryan Gosling) on a journey of self

{'input': 'What is a very quick summary of the plot of the Barbie movie?',
 'chat_history': [AIMessage(content='The Barbie movie is a 2023 American fantasy comedy film directed by Greta Gerwig. It is the first live-action Barbie film based on the Barbie fashion dolls by Mattel. The film follows Barbie (played by Margot Robbie) and Ken (played by Ryan Gosling) on a journey of self-discovery following an existential crisis. The movie explores themes of self-confidence and features an ensemble supporting cast, including America Ferrera, Kate McKinnon, Helen Mirren, Issa Rae, Simu Liu, Michael Cera, Rhea Perlman, and Will Ferrell.', additional_kwargs={}, example=False)],
 'output': 'The Barbie movie is a 2023 American fantasy comedy film directed by Greta Gerwig. It is the first live-action Barbie film based on the Barbie fashion dolls by Mattel. The film follows Barbie (played by Margot Robbie) and Ken (played by Ryan Gosling) on a journey of self-discovery following an existential crisis

### Oppenheimer Retrieval System

We're going to repourpose some of what we created previously, but this time we'll explore a different multi-source retrieval system.

In [17]:
oppenheimer_wikipedia_docs = WikipediaLoader(query="Oppenheimer (film)", load_max_docs=1, doc_content_chars_max=1_000_000).load()

oppenheimer_csv_docs = CSVLoader(file_path="./oppenheimer_data/oppenheimer.csv", source_column="Review_Url").load()

In [18]:
chunked_opp_wikipedia_docs = wikipedia_text_splitter.transform_documents(oppenheimer_wikipedia_docs)
chunked_opp_csv_docs = csv_text_splitter.transform_documents(oppenheimer_csv_docs)

In [103]:
opp_csv_faiss_retriever = FAISS.from_documents(chunked_opp_csv_docs, embedder).as_retriever()

# set up BM25 retriever
opp_wikipedia_bm25_retriever = BM25Retriever.from_documents(
    chunked_opp_wikipedia_docs
)

opp_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
opp_wikipedia_faiss_store = FAISS.from_documents(
    chunked_opp_wikipedia_docs, embedder
)

opp_wikipedia_faiss_retriever = opp_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
opp_ensemble_retriever = EnsembleRetriever(
    retrievers=[opp_wikipedia_bm25_retriever, opp_wikipedia_faiss_retriever],
    weights=[0.25, 0.75] # should sum to 1
)

#### Multi-source chain

We're going to allow the LLM to decide which information is most -> least valuable.

The way we'll do this is with LangChain's rather powerful "Expression Language"!

> HINT: You can leverage [this](https://python.langchain.com/docs/use_cases/question_answering/how_to/multiple_retrieval) resource if you get stuck - but experiment with different prompts/formats.

In [104]:
from langchain.prompts import ChatPromptTemplate

system_message = """Use the information from the below two sources to answer any questions.

Source 1: public user reviews about the Oppenheimer movie
<source1>
{source1}
</source1>

Source 2: the wikipedia page for the Oppenheimer movie including the plot summary, cast, and production information
<source2>
{source2}
</source2>
"""

prompt = ChatPromptTemplate.from_messages([("system", system_message), ("human", "{question}")])

In [105]:
oppenheimer_multisource_chain = {
    "source1": (lambda x: x["question"]) | opp_ensemble_retriever,
    "source2": (lambda x: x["question"]) | opp_csv_faiss_retriever,
    "question": lambda x: x["question"],
} | prompt | llm

In [106]:
oppenheimer_multisource_chain.invoke({"question" : "What did people think of the Oppenheimer movie?"})

AIMessage(content="People generally thought highly of the Oppenheimer movie. Critics praised the screenplay, performances of the cast (especially Cillian Murphy and Robert Downey Jr.), and the visuals. It was frequently cited as one of the best films of 2023. On the review aggregator website Rotten Tomatoes, 93% of 450 critics' reviews were positive, with an average rating of 8.6/10. The movie received critical acclaim, with particular praise for the cast performances, screenplay, and visuals. However, some criticism was aimed towards the writing of the female characters. Overall, the movie was considered compelling, engaging, and simple to understand, and it sparked an interest in history.", additional_kwargs={}, example=False)

# Agent Creation

Now we can finally start building our Agent!

The first thing we'll need to do is provide our Agent a Toolbelt. (list of tools). Much like Batman, our LLM-powered Agent can use these tools as it sees fit. 

While the examples we're constructing in this notebook are straightforward for brevity and simplicities sake - there is no limit to what you can build with Agents, as we'll see as we progress through the program.

So, let's begin by setting up our Tools!

You'll notice that we have to set up a function to allow our `OppenheimerInfo` tool to interface with the Agent - this is due to it have a specific required input. Creating custom tools is a pattern that you'll want to grow acustomed to as you use LangChain more and more.

In [107]:
from langchain.agents import Tool

def query_oppenheimer(input):
    return oppenheimer_multisource_chain.invoke({"question" : input})

tools = [
    Tool(
        name = "BarbieInfo",
        func=barbie_retriever_agent_executor.invoke,
        description="useful for when you need to answer questions about Barbie. Input should be a fully formed question."
    ),
    Tool(
        name = "OppenheimerInfo",
        func=query_oppenheimer,
        description="useful for when you need to answer questions about Oppenheimer. Input should be a fully formed question."
    ),
]

Now that we've set up our Agents toolbelt, let's set up the LLM that will be powering it!

I would suggest playing around with these prompts - and experiments to find what works best for you.

In [108]:
from langchain.agents import ZeroShotAgent, AgentExecutor

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

In [109]:
from langchain import LLMChain

llm_chain = LLMChain(llm=llm, prompt=prompt)

All that's left to do now is create our `ZeroShotAgent` and our `AgentExecutor`, which are the "reasoner" and "actor" halfs of the `ReAct` method of Agent implementation.

Read all about the `ReAct` framework [here](https://react-lm.github.io/)

In [110]:
barbenheimer_agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
barbenheimer_agent_chain = AgentExecutor.from_agent_and_tools(agent=barbenheimer_agent, tools=tools, verbose=True)

## Conclusion

All that is left to do now, is feed inputs to your Agent and watch it work!

Remember to use the `{"input" : "YOUR QUERY HERE"}` format when prompting the Agent.

In [97]:
barbenheimer_agent_chain.invoke({"input" : "What did people like about the Barbie movie?"})



> Entering new AgentExecutor chain...
Thought: I need to find information about the Barbie movie.
Action: BarbieInfo
Action Input: What did people like about the Barbie movie?

> Entering new AgentExecutor chain...
Based on the available reviews, people liked several aspects of the Barbie movie:

1. Visual Spectacle: The movie is praised for its wonderful visual spectacle, indicating that the visuals and cinematography were impressive.

2. Creative Physical Comedy: The movie is described as having a great deal of creative physical comedy, suggesting that the humor and comedic elements were well-received by viewers.

3. Empowerment Theme: Some viewers appreciated the theme of empowerment in the movie, which resonated with them and made the film a big hit.

4. Darkly Satirical Look at Gender Structure: The movie is commended for taking a darkly satirical look at gender structure in society, providing a unique and thought-provoking perspective on the subject.

5. Emotional Engagement: T

{'input': 'What did people like about the Barbie movie?',
 'output': 'People liked several aspects of the Barbie movie, including its visual spectacle, creative physical comedy, empowerment theme, darkly satirical look at gender structure, emotional engagement, funny comedy, music and choreographed dances, and nostalgic representation of the Barbie brand.'}

In [98]:
barbenheimer_agent_chain.run({"input" : "What did people like about the Oppenheimer movie?"})



> Entering new AgentExecutor chain...
Thought: I need to find out what people liked about the Oppenheimer movie.
Action: OppenheimerInfo
Action Input: "What did people like about the Oppenheimer movie?"
Observation: content='People liked the screenplay, performances of the cast (particularly those of Murphy and Downey), and the visuals of the Oppenheimer movie. They also praised the accuracy of the movie, with some scenes being taken word-for-word from the book or real-life events. The cinematography and practical effects were also praised.' additional_kwargs={} example=False
Thought:I now know what people liked about the Oppenheimer movie.
Final Answer: People liked the screenplay, performances of the cast, visuals, accuracy, cinematography, and practical effects of the Oppenheimer movie.

> Finished chain.


'People liked the screenplay, performances of the cast, visuals, accuracy, cinematography, and practical effects of the Oppenheimer movie.'

In [111]:
barbenheimer_agent_chain.run({"input" : "Did the movies Barbie and Oppenheimer share similar themes or ideas?"})



> Entering new AgentExecutor chain...
Thought: I need to find information about both Barbie and Oppenheimer to determine if their movies share similar themes or ideas.
Action: BarbieInfo
Action Input: Did Barbie movies have any specific themes or ideas?

> Entering new AgentExecutor chain...
Yes, Barbie movies often have specific themes and ideas. Some common themes in Barbie movies include friendship, adventure, self-discovery, empowerment, and the importance of being true to oneself. These movies often convey positive messages and teach valuable life lessons to young viewers. Additionally, Barbie movies often promote diversity, inclusivity, and the idea that anyone can achieve their dreams.

> Finished chain.

Observation: {'input': 'Did Barbie movies have any specific themes or ideas?', 'chat_history': [HumanMessage(content='Did Barbie movies have any specific themes or ideas?', additional_kwargs={}, example=False), AIMessage(content='Yes, Barbie movies often have specific themes 

"Yes, both Barbie movies and Oppenheimer's movies explore specific themes and ideas. Barbie movies often focus on friendship, adventure, self-discovery, empowerment, and the importance of being true to oneself. They also promote diversity, inclusivity, and the idea that anyone can achieve their dreams. On the other hand, Oppenheimer's movies delve into themes such as internal conflict, guilt, the red scare, McCarthyism, and the ethical implications of developing powerful weapons. While there may be some overlap in terms of exploring internal conflict and ethical dilemmas, the overall themes and ideas in Barbie movies and Oppenheimer's movies are quite different."

## Next Steps

It's time to build a Chainlit (or Gradio) application and host it on Hugging Face Spaces! :ship: